In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import import_ipynb
#import Segmentation
import time
import preds

importing Jupyter notebook from preds.ipynb
importing Jupyter notebook from Captcha_seg.ipynb
1080


/home/bg/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 864 samples, validate on 216 samples
Epoch 1/20
864/864 [==============================] - 26s 30ms/step - loss: 3.5933 - acc: 0.0289 - val_loss: 3.5872 - val_acc: 0.0139
Epoch 2/20
864/864 [==============================] - 14s 16ms/step - loss: 3.4477 - acc: 0.0995 - val_loss: 3.1846 - val_acc: 0.1389
Epoch 3/20
864/864 [==============================] - 14s 16ms/step - loss: 2.5782 - acc: 0.2894 - val_loss: 2.3189 - val_acc: 0.3519
Epoch 4/20
864/864 [==============================] - 13s 16ms/step - loss: 1.7350 - acc: 0.4803 - val_loss: 1.7741 - val_acc: 0.4861
Epoch 5/20
864/864 [==============================] - 13s 15ms/step - loss: 1.1989 - acc: 0.6285 - val_loss: 1.7228 - val_acc: 0.5648
Epoch 6/20
864/864 [==============================] - 12s 14ms/step - loss: 0.8424 - acc: 0.7419 - val_loss: 1.5405 - val_acc: 0.6065
Epoch 7/20
864/864 [==============================] - 12s 13ms/step - loss: 0.5624 - acc: 0.8252 - val_loss: 1.5386 - val_acc: 0.6157
Epoch 8/20
864/8

In [2]:
CATEGORIES = set()
names =[]
for i in os.listdir('test/'):
    names.append(i.split('.')[0])
for i in names:
    for x in list(i):
        CATEGORIES.add(x)
CATEGORIES=list(CATEGORIES)
print(CATEGORIES)

['t', 'h', 'e', 'i', 'd', 'j', 'l', 'r', 'o', 's', 'u', 'p', 'm', 'v', 'b', 'q', 'n', 'f', 'x', 'a', 'c', 'k', 'g', 'w', 'z', 'y']


In [3]:
#print(len(train_data))

In [4]:
import random

#random.shuffle(train_data)

In [5]:
#for sample in train_data[:10]:
#    print(sample[1])

In [6]:
X = []
y = []

for features, label in train_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

import pickle

pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()
pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_in= open("X.pickle", "rb")
X = pickle.load(pickle_in)
X.shape[1:]

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation, Flatten, Conv2D, MaxPooling2D

X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

X = X/255.0

model = Sequential()

model.add(Conv2D(64,(3,3),input_shape= X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))


model.add(Dense(19))
model.add(Activation("softmax"))

from keras.utils import to_categorical
y_binary = to_categorical(y)

model.compile(loss="categorical_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

model.fit(X,y_binary,batch_size=128, epochs=20, validation_split = 0.2)


In [7]:
#loss_value, metrics_value = model.evaluate(X,y_binary,batch_size=10)

In [8]:
#print(loss_value, metrics_value)

In [9]:
import random

In [10]:
def similarity(a,b):
    i =0
    for x,y in zip(a,b):
        if x==y:
            i+= 1
    percentage = (i/5) * 100
    return percentage

def prepare(filepath):
    IMG_SIZE = 50
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))
    #plt.imshow(new_array,'gray')
    return new_array.reshape(-1, IMG_SIZE,IMG_SIZE,1)

total_time = 0
score = 0
count = 0
done = []
list_ = preds.predictions('test',CATEGORIES)
CAPTCHA_DIR = list_[0]
for x in CAPTCHA_DIR:
    captchas = random.choice(CAPTCHA_DIR)
    if captchas in done:
        continue
    else:
        done.append(captchas)
        start_time = time.time()
        predict_list= []
        print('\n')
        print('Actual: ',captchas[0][0])

        files = []
        for segments in list_:
            files.append(segments)

        #for segments in files:
        #    filepath = (SEGMENTS_DIR,segments)
        #    prediction = model.predict([prepare(filepath)])
        #    predict_list.append(CATEGORIES[np.argmax(prediction[0])])

        elapsed_time = time.time()-start_time
        time.sleep(captchas[1])
        total_time += elapsed_time
        final_prediction = ''.join(predict_list)
        #if captcha_name[0]== final_prediction:
        #    score +=1
        score = list_[1]
        print('Predicted:',captchas[0][1],'\n')
        print('Elapsed time= ',captchas[1],'s')
        total_time +=captchas[1]
        print('<------------------------------->')    
    

print('Total number of CAPTCHAS broken: ',score)    
print('Total number of CAPTCHAS attempted: ',len(list_[0]))
print('Total time taken= ',total_time,'s')
print('Accuracy= ', (score/len(list_[0])*100))
#model = tf.keras.models.load_model('captcha_alpha.model')
#prediction = model.predict([prepare('3.png')])
#print(len(prediction[0]))
#print(prediction[0])
#print(CATEGORIES[np.argmax(prediction[0])])



Actual:  slabshacks
Predicted: hlxs 

Elapsed time=  0.0050348693829631586 s
<------------------------------->


Actual:  yokotides
Predicted: yokotides 

Elapsed time=  0.28001962310314576 s
<------------------------------->


Actual:  tackyinky
Predicted: tackyinky 

Elapsed time=  0.6775579584510206 s
<------------------------------->


Actual:  arubachief
Predicted: arubachief 

Elapsed time=  0.034257535539099715 s
<------------------------------->


Actual:  ulnasharts
Predicted: ulnasharts 

Elapsed time=  0.1340628961252439 s
<------------------------------->


Actual:  visaovid
Predicted: visaovid 

Elapsed time=  0.7958339496003894 s
<------------------------------->


Actual:  paulisnit
Predicted: paulisnit 

Elapsed time=  0.6299161696675812 s
<------------------------------->


Actual:  hacksbarth
Predicted: hacksbarth 

Elapsed time=  0.32073794373425335 s
<------------------------------->


Actual:  gettyaryan
Predicted: gettyaryan 

Elapsed time=  0.04020401966728582 

Predicted: aortabeige 

Elapsed time=  0.36944046269985664 s
<------------------------------->


Actual:  ruingturfs
Predicted: ruingturfs 

Elapsed time=  0.1526728411264897 s
<------------------------------->


Actual:  coltsbrad
Predicted: coltsbrad 

Elapsed time=  0.6401803951432299 s
<------------------------------->


Actual:  daredthais
Predicted: haeh 

Elapsed time=  0.11282277359021309 s
<------------------------------->


Actual:  creexscold
Predicted: creexscold 

Elapsed time=  0.6953051413110687 s
<------------------------------->


Actual:  rillsabeam
Predicted: rillsabeam 

Elapsed time=  0.9525116931933805 s
<------------------------------->


Actual:  monthbani
Predicted: monthbani 

Elapsed time=  0.3569553461868522 s
<------------------------------->


Actual:  knowspurse
Predicted: knowspurse 

Elapsed time=  0.10686922567273693 s
<------------------------------->


Actual:  dragindue
Predicted: dragindue 

Elapsed time=  0.665856759958474 s
<---------------------

Predicted: blairkale 

Elapsed time=  0.20690753433573206 s
<------------------------------->


Actual:  visaovid
Predicted: visaovid 

Elapsed time=  0.3622926392478699 s
<------------------------------->


Actual:  aortabeige
Predicted: aortabeige 

Elapsed time=  0.007420664516835318 s
<------------------------------->


Actual:  bandgamed
Predicted: bandgamed 

Elapsed time=  0.638726056866866 s
<------------------------------->


Actual:  rearmhayed
Predicted: rearmhayed 

Elapsed time=  0.7180159773380527 s
<------------------------------->


Actual:  idolsets
Predicted: idolsets 

Elapsed time=  0.06454620646697651 s
<------------------------------->


Actual:  alignkeri
Predicted: alignkeri 

Elapsed time=  0.5495423300712223 s
<------------------------------->


Actual:  hazesbarfs
Predicted: hazesbarfs 

Elapsed time=  0.7114961456375305 s
<------------------------------->


Actual:  songlents
Predicted: songlents 

Elapsed time=  0.7638499635278784 s
<-----------------------

Predicted: ogleprowl 

Elapsed time=  0.6193550918923635 s
<------------------------------->


Actual:  lavaroach
Predicted: lavaroach 

Elapsed time=  0.4970975459659157 s
<------------------------------->


Actual:  ejectkrebs
Predicted: ejectkrebs 

Elapsed time=  0.6262672503245187 s
<------------------------------->


Actual:  mobsdiego
Predicted: mobsdiego 

Elapsed time=  0.42565798792674114 s
<------------------------------->


Actual:  tenongonna
Predicted: tenongonna 

Elapsed time=  0.556623651538452 s
<------------------------------->


Actual:  inaptaugur
Predicted: inaptaugur 

Elapsed time=  0.6858769133282362 s
<------------------------------->


Actual:  rigeldims
Predicted: rigeldims 

Elapsed time=  0.11260690479736613 s
<------------------------------->


Actual:  wormslevy
Predicted: wormslevy 

Elapsed time=  0.2816092858342334 s
<------------------------------->


Actual:  barnstipis
Predicted: barnstipis 

Elapsed time=  0.2952753193352462 s
<-------------------